In [1]:
import numpy as np
from exputils.dot.dot_product import compute_all_dot_products
from exputils.RoM.dot import calculate_RoM_dot
from exputils.state.random import make_random_quantum_state

Calculate purity from Pauli vector

In [2]:
from exputils.state.random import (
    _make_random_mixed_density_matrix,
    _make_random_pure_density_matrix,
    state_in_pauli_basis,
)


def purity(n_qubit, rho_vec):
    return np.dot(rho_vec, rho_vec) / 2**n_qubit


for n_qubit in range(1, 5 + 1):
    # pure
    for seed in range(100):
        pure_dm = _make_random_pure_density_matrix(n_qubit, seed)
        purity_from_dm = np.trace(pure_dm @ pure_dm)
        pure_pv = state_in_pauli_basis(pure_dm)
        purity_from_pv = purity(n_qubit, pure_pv)
        assert abs(purity_from_dm - purity_from_pv) < 10**-8
        assert abs(purity_from_dm - 1) < 10**-8
    # mixed
    for seed in range(100):
        mixed_dm = _make_random_mixed_density_matrix(n_qubit, seed)
        purity_from_dm = np.trace(mixed_dm @ mixed_dm)
        mixed_pv = state_in_pauli_basis(mixed_dm)
        purity_from_pv = purity(n_qubit, mixed_pv)
        assert abs(purity_from_dm - purity_from_pv) < 10**-8
    print(n_qubit, "ok")

1 ok
2 ok
3 ok
4 ok
5 ok


In [3]:
def lb_by_dot(n_qubit, rho_vec):
    dots = compute_all_dot_products(n_qubit, rho_vec)
    dp = np.max(dots)
    dm = np.min(dots)
    assert np.isclose(rho_vec.T @ rho_vec, np.linalg.norm(rho_vec, ord=2) ** 2)
    return 1 + 2 * max(0, (np.linalg.norm(rho_vec, ord=2) ** 2 - dp) / (dp - dm))


def lb_by_stabilizer_fidelity(n_qubit, rho_vec):
    dots = compute_all_dot_products(n_qubit, rho_vec)
    stabilizer_fidelity = np.max(dots) / (2**n_qubit)
    return purity(n_qubit, rho_vec) / stabilizer_fidelity

In "Simulation of quantum circuits by low-rank stabilizer decompositions"

(Sergey Bravyi, Dan Browne, Padraic Calpin, Earl Campbell, David Gosset, and Mark Howard)

It is said that $F(T) \approx 0.853$.

We confirm this.


In [4]:
from exputils.state.tensor import make_random_tensor_product_state

n_qubit = 1
rho_vec = make_random_tensor_product_state("H", n_qubit, seed=0)
dots = compute_all_dot_products(n_qubit, rho_vec)
stabilizer_fidelity = np.max(dots) / (2**n_qubit)
print(stabilizer_fidelity)

0.8535533905932737


In [5]:
for n_qubit in range(1, 4 + 1):
    print(n_qubit)
    for kind in ["pure", "mixed"]:
        print("=" * 10, f"{kind=}", "=" * 10)
        for seed in range(10):
            rho_vec = make_random_quantum_state(kind, n_qubit, seed=seed)
            RoM, coeff, Amat = calculate_RoM_dot(n_qubit, rho_vec, method="gurobi")
            lb_sf = lb_by_stabilizer_fidelity(n_qubit, rho_vec)
            print(f"{lb_sf=} {RoM=} {lb_sf <= RoM=}")
            assert lb_sf <= RoM

1
========== kind='pure' ==========
Set parameter Username
Academic license - for non-commercial use only - expires 2024-06-16
lb_sf=1.1411099356282524 RoM=1.5451005656365662 lb_sf <= RoM=True
lb_sf=1.170615797495209 RoM=1.6645578485498636 lb_sf <= RoM=True
lb_sf=1.0450586802552697 RoM=1.423146437112564 lb_sf <= RoM=True
lb_sf=1.0598271805699049 RoM=1.5353615356419994 lb_sf <= RoM=True
lb_sf=1.0940137279323066 RoM=1.4695565407451046 lb_sf <= RoM=True
lb_sf=1.1208764815663035 RoM=1.6421580587760896 lb_sf <= RoM=True
lb_sf=1.0895762686275636 RoM=1.6121758902757246 lb_sf <= RoM=True
lb_sf=1.1614754261546862 RoM=1.5822233576575728 lb_sf <= RoM=True
lb_sf=1.1082210427085017 RoM=1.642822070155682 lb_sf <= RoM=True
lb_sf=1.2237749963500277 RoM=1.7203677972688054 lb_sf <= RoM=True
========== kind='mixed' ==========
lb_sf=1.0432828705438035 RoM=1.393184077027386 lb_sf <= RoM=True
lb_sf=0.940893371829701 RoM=1.0 lb_sf <= RoM=True
lb_sf=0.9178081867498779 RoM=0.9999999999999999 lb_sf <= RoM=True
